In [4]:
#!jupyter nbconvert --to script web_spider4_down_pdf.ipynb

import requests
#url = 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2010-03-31/600795_2009_n.pdf'
# 对于szse.cn 有点特殊，有的用download可以下载，有的不能
url = 'http://disc.static.szse.cn/download/disc/disk01/finalpage/2008-04-19/cda5e777-cc3c-4934-9971-99ce9e4f6fe0.PDF'
url = 'http://disc.static.szse.cn/disc/disk01/finalpage/2008-04-19/cda5e777-cc3c-4934-9971-99ce9e4f6fe0.PDF'
def down_pdf(filename,url):
    success = True
    try:
        r = requests.get(url, stream=True)
        with open(filename, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=102400):
                fd.write(chunk)
    except Exception as e:
        success = False
        print(f'download {url} faild',e)
        pass
    
    return filename,success

import subprocess
import os 
import sys
def execute(command):
#     print(command," ".join(command))
#     " ".join(command)
    process = subprocess.Popen("  ".join(command), shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    # Poll process for new output until finished
    while True:
        if process.poll() is not None:
            break
        nextline = process.stdout.readline()
        if type(nextline) is bytes:
            nextline = nextline.decode("utf-8") 
        sys.stdout.write(nextline)
        sys.stdout.flush()
        if process.poll() is not None:
            break

    output = process.communicate()[0]
    exitCode = process.returncode

    if (exitCode == 0):
        return output
    else:
        raise ProcessException(command, exitCode, output)

def down_pdf2(filename,url):

    success = True
    try:
        if os.path.exists(filename):
            os.remove(filename)
        # --no-use-server-timestamps  不根据服务器上的时间戳设置本地文件的时间戳
        execute(["wget","--no-use-server-timestamps","-O",filename ,url])
    except Exception as e:
        success = False
        print(f'download {url} faild',e)
        pass
    
    return filename,success

# down_pdf2("xx.pdf",url)

In [5]:
# 检查文件合法性
import os
from PyPDF2 import PdfFileWriter, PdfFileReader
def checkPdf(f):
    """检查pdf文件的合法性"""
    # 文件是否存在
    if not os.path.exists(f):
        return False
    
    # 文件大小是否满足条件
    fsize=0
    try:
        fsize = os.path.getsize(f)
    except Exception as e:
        print(f'getsize exception {f} ',e)
        return False

    if fsize < 1024*50:
        return False
    
    # 是否正常的pdf文件
    try:
        pdfFileReader = PdfFileReader(f)
        if pdfFileReader.isEncrypted:
            pdfFileReader.decrypt('')
        if pdfFileReader.getDocumentInfo()  is not None:
            return True
    except:
        pass
    return False
    

In [9]:
import time
import json
from tqdm import tqdm

# python  down_pdf.py 2013 3 1 表示启动三个进程，除以3取摸为1的进行
# 一年一年下载，然后逐个检查
import sys
import os
YEAR = '2017'
a1,a2 =1,0
is_reverse = False
if sys.argv[0].find("ipykernel_launcher.py")<=0:
    if len(sys.argv)>=2:
        YEAR = sys.argv[1]
    if len(sys.argv)>=4:
        a1=int(sys.argv[2])
        a2=int(sys.argv[3])
    if len(sys.argv)>=5:
        is_reverse = True
 
with open('pdf_status_with_url.json') as json_file:  
    pdf_urls = json.load(json_file)

_keys =[]    
for code_year,data in pdf_urls.items():
    code,year = code_year.split('_')
    if data[1]>20 or data[0]>500:
        continue    
    _keys.append(code_year)

keys = list(filter(lambda x:YEAR=='all' or x[-4:] in YEAR,_keys))
keys = sorted(keys,key=lambda x:x[-4:],reverse=True)
print(f'pdf_urls={len(pdf_urls)},_keys={len(_keys)},this process {len(keys)}')
if is_reverse>4:
    keys.reverse()
    
def down_loop():
    pdf_dir ='../pdf/'
    for code_year in tqdm(keys):
        code,year = code_year.split('_')
        if int(code) % a1 !=a2:
#             print('jump',code)
            continue

        pdf_file=os.path.join(pdf_dir,year,code_year+'.pdf')
        if checkPdf(pdf_file):
            print(f'{pdf_file} is ok,jump...')
            continue

        times = 0
        while times<= 3:
            times +=1
            t0=time.time()
            data = pdf_urls[code_year]
            if len(data)>=3:
                url = data[2]
            else:
                print('没有url，code_year=',code_year)
            # 深圳交易所比较特殊，有些情况下不需要加download反而更快
            if times%2 == 1:
                url = url.replace('szse.cn/download/','szse.cn/')
            print(f'开始下载文件：{url},times={times}')
            pdf_file,success = down_pdf2(pdf_file,url)
            if not success:
                time.sleep(2)
                continue
            fsize=1
            try:
                fsize = os.path.getsize(pdf_file)
            except:
                pass
            print(f'下载文件({pdf_file})成功，用时(秒):{time.time()-t0},times={times},fsize={fsize/1024}KB')
            t0=time.time()
            if checkPdf(pdf_file):
                print('+'*30,f'校验文件({pdf_file})成功')
                break
            else:
                print('-'*30,f'校验文件({pdf_file})失败，times={times}')
    
    time.sleep(120)
    
    
down_loop()
    




  0%|          | 0/12 [00:00<?, ?it/s]

pdf_urls=30555,_keys=2508,this process 12
开始下载文件：http://disc.static.szse.cn/disc/disk01/finalpage/2018-04-24/8b27501e-bb25-4b67-a199-6dd5b2a45ee0.PDF,times=1
--2019-06-02 15:31:12--  http://disc.static.szse.cn/disc/disk01/finalpage/2018-04-24/8b27501e-bb25-4b67-a199-6dd5b2a45ee0.PDF
正在解析主机 disc.static.szse.cn (disc.static.szse.cn)... 115.231.23.136
正在连接 disc.static.szse.cn (disc.static.szse.cn)|115.231.23.136|:80... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度： 2469398 (2.4M) [application/pdf]
正在保存至: “../pdf/2017/002585_2017.pdf”

     0K .......... .......... .......... .......... ..........  2% 43.1M 0s


KeyboardInterrupt: 

In [3]:
fileName='123456_2005'
code,year = fileName.split('_')
print(code,year)
print('2005' in fileName)
print('2005' in '2005')

123456 2005
True
True
